In [ ]:
import os
import numpy as np
import pandas as pd, gc
import matplotlib.pyplot as plt
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

import cudf, cuml, cupy
from cuml.neighbors import NearestNeighbors

from tqdm import tqdm
# timm.list_models(pretrained=True)

In [ ]:
# Constant
TRAIN_PATH="../input/shopee-product-matching/train.csv"
TRAIN_SIAMESE_PATH="../input/code-for-data-generation-for-siamese-training/siamese_data.csv"
TEST_PATH="../input/shopee-product-matching/test.csv"
TRAIN_IMAGE_PATH="../input/shopee-product-matching/train_images/" # + image id
TEST_IMAGE_PATH="../input/shopee-product-matching/test_images/"

'''
efficientnet_b0
efficientnet_b4
efficientnetv2_rw_m
vgg16
swin_large_patch4_window12_384
gluon_resnext101_32x4d
'''
BASE_MODEL = "efficientnet_b0"

DIM = (512, 512)
# DIM = (384, 384)
# DIM = (320, 320)
# DIM = (256, 256)
N_CLASS = 11014

BATCH_SIZE=32

COMPUTE_CV = False

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        torch.backends.cudnn.benchmark = True
    else:
        device = torch.device('cpu') # don't have GPU 
    return device
DEVICE = get_device()
print(DEVICE)

In [ ]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

train_df['target'] = train_df.label_group.map(
    train_df.groupby('label_group').posting_id.agg('unique').to_dict()
)

N_CLASS = train_df['label_group'].nunique()
print("n_trian: {} n_unique: {} n_per_images: {}"
         .format(train_df['label_group'].shape[0],
                 train_df['label_group'].nunique(),
                 train_df['label_group'].shape[0]/train_df['label_group'].nunique())
         )

sample_img = cv2.imread(TRAIN_IMAGE_PATH+train_df.loc[0, 'image'])
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
# plt.imshow(sample_img)
# plt.show()
train_df.head()

In [ ]:
'''Transform only image'''
def train_transforms():
    return A.Compose([
        A.Normalize(
            max_pixel_value=255.0, always_apply=True
        ),
        ToTensorV2()
    ])

x = train_transforms()(image=sample_img)['image']
print(x.shape, x.min(), x.max())

In [ ]:
tmp = train_df.groupby('image_phash').posting_id.agg('unique').to_dict()
train_df['oof'] = train_df.image_phash.map(tmp)
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score
train_df['f1'] = train_df.apply(getMetric('oof'),axis=1)
print('CV score for baseline =',train_df.f1.mean())

In [ ]:
if COMPUTE_CV:
    test = pd.read_csv('../input/shopee-product-matching/train.csv')
    test_gf = cudf.DataFrame(test)
    print('Using train as test to compute CV (since commit notebook). Shape is', test_gf.shape )
else:
    test = pd.read_csv('../input/shopee-product-matching/test.csv')
    test_gf = cudf.read_csv('../input/shopee-product-matching/test.csv')
    print('Test shape is', test_gf.shape )
test_gf.head()

In [ ]:
class ImageModel(nn.Module):
    def __init__(self, model_name=BASE_MODEL, pretrained=True):
        self.model_name = model_name
        self.pretrained=pretrained
        super().__init__()
        
        if model_name=="resnet50":
            self.backbone_model = torchvision.models.resnet50(pretrained=True)
        
        else:
            self.backbone_model = timm.create_model(model_name, pretrained=self.pretrained)
            if not self.pretrained:
                checkpoint_path = '../input/newefficentnetb0weights/efficientnet_b0_ra-3dd342df.pth'
                pretrained_weights = torch.load(checkpoint_path)
#                 print(pretrained_weights)
                self.backbone_model.load_state_dict(pretrained_weights)
            
        
    def forward(self, image):
        output = self.backbone_model(image)
        return output

In [ ]:
class ShopeeDataset(Dataset):
    def __init__(self, image,labels,dim=DIM, augmentation=None, is_train=True):
        self.image = image
        self.labels = labels
        self.dim = dim
        self.is_train = is_train
        self.augmentation = augmentation
        
    def get_image(self, image_path, is_train=True):
        if self.is_train:
            img = cv2.imread(os.path.join(TRAIN_IMAGE_PATH, image_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.dim)
        else:
            img = cv2.imread(os.path.join(TEST_IMAGE_PATH, image_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.dim)
        return img
        
    def __len__(self):
        return len(self.image)
    
    def __getitem__(self, idx):
        img = self.image[idx]
        img = self.get_image(img)
        
        if self.augmentation:
            tmp = self.augmentation(image=img)
            img = tmp['image']
            
        return img
        

In [ ]:
# BASE = '../input/shopee-product-matching/test_images/'
# if COMPUTE_CV: BASE = '../input/shopee-product-matching/train_images/'

model = ImageModel(pretrained=False)
model = model.to(DEVICE)
embeds = []
# print(len(train_df['label_group'].values.tolist()))

if COMPUTE_CV:
    train_ds = ShopeeDataset(
        image = train_df['image'].values.tolist(),
        labels = 0,
        dim = DIM,
        is_train = True,
        augmentation=train_transforms(),
    )
else:
    train_ds = ShopeeDataset(
        image = test_df['image'].values.tolist(),
        labels = 0,
        dim = DIM,
        is_train = False,
        augmentation=train_transforms(),
    )
    print(COMPUTE_CV, test_df['image'].values.tolist())

train_dl = DataLoader(
    train_ds,
    batch_size=32,
#     pin_memory=True,
    num_workers=2
)

# for e,  in train_dl:
#     print(e)

print('Computing image embeddings...')
for i, tmp in enumerate(tqdm(train_dl)):
    with torch.no_grad():
        tmp = tmp.to(DEVICE)
        image_embeddings = model(tmp)
        embeds.append(image_embeddings.cpu())

image_embeddings = np.concatenate(embeds)
del train_ds, train_dl, embeds, model
_ = gc.collect()
print('image embeddings shape',image_embeddings.shape)

In [ ]:
KNN = 50
if len(test)==3: KNN = 2
model = NearestNeighbors(n_neighbors=KNN)
model.fit(image_embeddings)

===============

In [ ]:
!pip install ../input/fairseq0102/sacrebleu-2.0.0-py3-none-any.whl
!pip install ../input/importlib523/importlib_resources-5.2.3-py3-none-any.whl
!pip install ../input/testtesttest/antlr4_python3_runtime-4.8-py3-none-any.whl
!pip install ../input/fairseq0102/omegaconf-2.1.2-py3-none-any.whl
!pip install ../input/fairseq0102/hydra_core-1.1.2-py3-none-any.whl
!pip install ../input/fairseq0102/fairseq-0.10.2-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
import math
from fairseq.models.roberta import RobertaModel

In [ ]:
EPOCH_NUM = 0
LR = 1e-05
BATCH_SIZE = 32
BATCH_PER_EPOCH = 1000

In [ ]:
roberta = RobertaModel.from_pretrained('../input/roberta-mnli-weights/roberta.large.mnli')

In [ ]:
roberta.cuda()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(roberta.model.classification_heads["mnli"].parameters(), lr=LR)

In [ ]:
max_tensor_size = 500

def convert(sent):
    res = torch.zeros(max_tensor_size - 1, dtype=torch.int64, device=DEVICE)
    t = roberta.encode(sent).to(DEVICE)
    res[0] = t.shape[0]
    res[1:t.shape[0] + 1] = t
    return res

temp_df = pd.read_csv("../input/shopee-product-matching/train.csv")

class MyDataset(Dataset):
    
    def __init__(self):
        super().__init__()
        self.df_length = temp_df.shape[0]
        self.data = torch.zeros(self.df_length, max_tensor_size, dtype=torch.int64, device=DEVICE)
        self.data[:, 0] = torch.tensor(temp_df["label_group"], dtype=torch.int64, device=DEVICE)
        
        temp = temp_df["title"].map(convert).to_numpy()
        for i, row in enumerate(temp):
            self.data[i, 1:max_tensor_size] = row
    
    def __getitem__(self, idx):
        first_idx = idx // self.df_length
        second_idx = idx % self.df_length
        return self.get_features(first_idx, second_idx)
    
    def get_features(self, idx1, idx2):
        target = 2 if self.data[idx1, 0] == self.data[idx2, 0] else 1
        first_len = self.data[idx1, 1]
        second_len = self.data[idx2, 1]
        x = torch.empty(first_len + second_len, dtype=torch.int64, device=DEVICE)
        x[:first_len] = self.data[idx1, 2:first_len + 2]
        x[first_len] = 2
        x[first_len + 1:] = self.data[idx2, 3:second_len + 2]
        return (roberta.extract_features(x)[:,0,:].detach().reshape(1, 1, -1), target)
    
    def __len__(self):
        return self.df_length ** 2

dataset = MyDataset()

In [ ]:
def valid(epoch, samples_set):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    print("validating...")
    for sample_idx in tqdm(range(samples_set.shape[0])):
        sample_idx %= dataset.df_length
        
        X, target = dataset.get_features(int(samples_set[sample_idx, 0]), int(samples_set[sample_idx, 1]))
        target = 1 if target == 2 else 0
        prediction = roberta.model.classification_heads["mnli"](X).argmax().item()
        prediction = 1 if prediction == 2 else 0
        
        if target == 0:
            if target == prediction:
                tn += 1
            else:
                fp += 1
        elif target == 1:
            if target == prediction:
                tp += 1
            else:
                fn += 1  
                
    accuracy = (tp + tn) / test_size
    recall = (tp / (tp + fp)) if (tp + fp) != 0 else 0
    precision = (tp / (tp + fn)) if (tp + fn) != 0 else 0
    f_score = (2 * recall * precision / (recall + precision)) if (recall + precision) != 0 else 0
    print("Epoch (" + str(epoch) + ")", end=" ")
    print("Accuracy :", accuracy, end=" ")
    print("Recall :", recall, end=" ")
    print("Precision :", precision, end=" ")
    print("F Score :", f_score)

In [ ]:
loss_history = []
will_valid = False
test_size = math.floor(dataset.df_length * 0.1)

for epoch in range(EPOCH_NUM):
    total_loss = 0
    
    samples_set = np.random.randint(0, high=dataset.df_length, size=(BATCH_SIZE * BATCH_PER_EPOCH, 2))

    current_sample = 0
    for batch in range(BATCH_PER_EPOCH):
        Xs = torch.empty(BATCH_SIZE, 1, 1024, device=DEVICE)
        targets = torch.zeros(BATCH_SIZE, 3, device=DEVICE)
        for i in range(BATCH_SIZE):
            X, y = dataset.get_features(int(samples_set[current_sample, 0]), int(samples_set[current_sample, 1]))
            current_sample += 1
            if current_sample >= len(dataset):
                current_sample = 0
            Xs[i][:] = X
            targets[i][y] = 1
        
        optimizer.zero_grad()
        predictions = roberta.model.classification_heads["mnli"](Xs)
        
        loss = loss_fn(predictions, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if will_valid:
        test_samples_set = np.random.randint(0, high=dataset.df_length, size=(test_size, 2))
        valid(epoch, test_samples_set)
    loss_history.append(total_loss)
    print("EPOCH loss :", total_loss)

In [ ]:
print(loss_history)

========
MERGER

In [ ]:
def predict_nlp(idx1, idx2):
    encoded = roberta.encode(test["title"].iloc[idx1], test["title"].iloc[idx2])
    features = roberta.extract_features(encoded)
    res = roberta.model.classification_heads["mnli"](features).argmax().item()
    return res == 2

In [ ]:
def predict(TS=18.0, print_data=True):
    preds = []
    CHUNK = 1024

    
    print('Finding similar images...')
    CTS = len(image_embeddings)//CHUNK
    if len(image_embeddings)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(image_embeddings))
        if print_data: print('chunk',a,'to',b)
        distances, indices = model.kneighbors(image_embeddings[a:b,])

        for k in range(b-a):
            IDX = np.where(distances[k,]<TS)[0]
            IDS = indices[k,IDX]
            
            temp = []
            for i in range(IDS.shape[0]):
                current = IDS[i]
                if predict_nlp(len(preds), current):
                    temp.append(current)
            
            
            IDS = np.array(temp)
            o = test.iloc[IDS].posting_id.values
            preds.append(o)
        
    return preds

preds = predict(30)
# preds
# del model, distances, indices, image_embeddings, embeds
# _ = gc.collect()
# best ts for efficientnet_b0: 18.0 f1_score: 0.6358098027565081 (512, 512)
# best ts for efficientnet_b0: 24.0 f1_score: 0.6201323875459158 (256, 256)
# best ts for efficientnetv2_rw_m: 13.5 f1_score: 0.5539241111324119 (320, 320) from paper
# best ts for efficientnetv2_rw_m: 12.0 f1_score: 0.5642019980069831 (512, 512)
# best ts for vgg16: 17.0 f1_score: 0.6318011447485347 (512, 512)

In [ ]:
test['preds'] = preds
tmp = [e.shape[0] for e in test['preds']]
print(max(tmp))
test.head()

In [ ]:
def combine_for_sub(row):
    x = np.concatenate([row.preds,row.preds2, row.preds3])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds,row.preds2, row.preds3])
    return np.unique(x)

def my_combine_for_sub(row):
    x = row.preds
    return ' '.join( np.unique(x) )

def my_for_cv(row):
    return row.preds

In [ ]:
def get_score(combine=my_for_cv):
    if COMPUTE_CV:
        tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
        test['target'] = test.label_group.map(tmp)
        test['oof'] = test.apply(combine,axis=1)
        test['f1'] = test.apply(getMetric('oof'),axis=1)
        print('CV Score =', test.f1.mean() )

get_score()

In [ ]:
def something():
    if COMPUTE_CV:
        log = []
        index_log = []
        for i in np.arange(9.0, 30.0, 0.5):
            test['preds'] = predict(i, print_data=False)
            tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
            test['target'] = test.label_group.map(tmp)
            test['oof'] = test.apply(my_for_cv,axis=1)
            test['f1'] = test.apply(getMetric('oof'),axis=1)
            print('TS = {},CV Score = {}'.format(i, test.f1.mean()))
            index_log.append(i)
            log.append(test.f1.mean())
    
something()

In [ ]:
test['matches'] = test.apply(my_combine_for_sub,axis=1)
test[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()